# Synthetic gradients for REINFORCE

## Introduction

In this homework, I'm making an attempt to the reduce variance of the **REINFORCE** algorithm. The vanilla update rule has the following form:

$$ 
    \theta_{t + 1} = \theta_{t} + \alpha \left( G_t - b(S_t) \right) \nabla_{\theta} \log \pi(A_t \, | \, S_t, \theta) \, ,
$$

where $ b(S_t) $ is a baseline function. The update above corresponds to a one-sample gradient estimate of the performance metric $ \eta(\theta) = v_{\pi_\theta}(s_0) $:

$$ 
    \nabla \eta(\theta) = \mathbb{E}_\pi \left[ G_t \nabla_{\theta} \log \pi(A_t \, | \, S_t, \theta) \right] \, .
$$

One possible way of reducing variance is to introduce an additional network estimating

$$ 
    \mathbb{E}_\pi \left[ G_t \nabla_{\theta} \log \pi(A_t \, | \, S_t, \theta) \, | \, S_t \right] \, .
$$

If we somehow managed to train it then we could use its outputs as a source of the policy gradient at each state encountered by the agent. This synthetic gradient is presumably less noisy as it incorporates information about past experiences.

I'm guessing, for a similar purpose we could also construct an estimate for the conditional expectation $ \mathbb{E}_\pi \left[ \cdot \, | \, S_t, A_t \right] $ but I'm not doing it here.

In [1]:
%matplotlib inline

In [2]:
import collections
import gym
import itertools
import matplotlib
import numpy as np
import os
import pandas as pd
import signal
import sys
import tensorflow as tf
import tensorflow.contrib.slim as slim

from functools import partial
from joblib import Parallel, delayed
from matplotlib import pyplot as plt

from lib import plotting
from lib.envs.cliff_walking import CliffWalkingEnv

matplotlib.style.use('ggplot')

## Define main classes and funcitons

In [3]:
def apply_synthetic_grads(policy, decay_factor=0.995, learning_rate=0.2, scope="apply_synthetic_grads"):
    with tf.variable_scope(scope):
        state_one_hot = tf.one_hot(policy.state, int(env.observation_space.n))
        pred = tf.contrib.layers.fully_connected(
            inputs=tf.expand_dims(state_one_hot, 0),
            num_outputs=env.action_space.n,
            activation_fn=None,
            weights_initializer=tf.zeros_initializer())
        
        # Compute REINFORCE gradient.
        target = tf.gradients(policy.loss, policy.output_layer)[0]
        target = tf.stop_gradient(target)

        # We push synthetic predictions to the REINFORCE estimate.
        loss = 0.5 * tf.reduce_sum(tf.square(pred - target))

        # Compute surrogate policy loss that has known gradients.
        pred = tf.stop_gradient(pred)
        alpha = tf.Variable(1.0, trainable=False)
        alpha = tf.assign(alpha, decay_factor * alpha)
        policy_loss = tf.reduce_sum(tf.multiply(policy.output_layer, alpha * target + (1.0 - alpha) * pred))
        
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
        train_op = slim.learning.create_train_op(
            loss, optimizer, global_step=tf.contrib.framework.get_global_step())
        
        return policy_loss, train_op

In [4]:
class Policy():    
    def __init__(self, learning_rate=0.01, loss_modifier=None, scope="policy"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.int32, [], "state")
            self.action = tf.placeholder(dtype=tf.int32, name="action")
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is just table lookup estimator
            state_one_hot = tf.one_hot(self.state, int(env.observation_space.n))
            self.output_layer = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(state_one_hot, 0),
                num_outputs=env.action_space.n,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer())

            self.action_probs = tf.squeeze(tf.nn.softmax(self.output_layer))
            self.picked_action_prob = tf.gather(self.action_probs, self.action)

            # Loss and train op
            self.loss = -tf.log(self.picked_action_prob) * self.target
            
            # Hack policy learning.
            if loss_modifier is not None:
                self.loss, loss_modifier_train_op = loss_modifier(self)
            else:
                loss_modifier_train_op = None

            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op = slim.learning.create_train_op(
                self.loss, self.optimizer, global_step=tf.contrib.framework.get_global_step())
        
            if loss_modifier_train_op is not None:
                self.train_op = self.train_op + loss_modifier_train_op
    
    def predict(self, state, sess=None):
        sess = sess or tf.get_default_session()
        return sess.run(self.action_probs, { self.state: state })

    def update(self, state, target, action, sess=None):
        sess = sess or tf.get_default_session()
        feed_dict = { self.state: state, self.target: target, self.action: action  }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [5]:
class ValueFunction():
    def __init__(self, learning_rate=0.1, scope="value_function"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.int32, [], "state")
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is just table lookup estimator
            state_one_hot = tf.one_hot(self.state, int(env.observation_space.n))
            self.output_layer = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(state_one_hot, 0),
                num_outputs=1,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer())

            self.value_estimate = tf.squeeze(self.output_layer)
            self.loss = tf.squared_difference(self.value_estimate, self.target)

            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())        
    
    def predict(self, state, sess=None):
        sess = sess or tf.get_default_session()
        return sess.run(self.value_estimate, { self.state: state })

    def update(self, state, target, sess=None):
        sess = sess or tf.get_default_session()
        feed_dict = { self.state: state, self.target: target }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [6]:
def reinforce(env, policy, value_function, num_episodes, epsilon=0.1, discount_factor=1.0):
    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
    try:
        for i_episode in range(num_episodes):
            # Reset the environment and pick the fisrst action
            state = env.reset()

            episode = []

            # One step in the environment
            for t in itertools.count():

                # Take a step
                action_probs = policy.predict(state)
                num_actions = len(action_probs)
                action = np.random.choice(np.arange(num_actions), p=action_probs)
                next_state, reward, done, _ = env.step(action)

                # Keep track of the transition
                episode.append(Transition(
                    state=state, action=action, reward=reward, next_state=next_state, done=done))

                # Update statistics
                stats.episode_rewards[i_episode] += reward
                stats.episode_lengths[i_episode] = t + 1

                if done:
                    break

                state = next_state

            # Go through the episode and make policy updates
            for t, transition in enumerate(episode):
                # The return after this timestep
                total_return = sum(discount_factor**i * t.reward for i, t in enumerate(episode[t:]))
                # Update our value estimator
                value_function.update(transition.state, total_return)
                # Calculate baseline/advantage
                baseline_value = value_function.predict(transition.state)            
                advantage = total_return - baseline_value
                # Update our policy estimator
                policy.update(transition.state, advantage, transition.action)
    except Exception, e:
        print e
    
    return stats

## Cliff walking experiment

I'm basing my experiments on the [policy gradient implementation by Denny Britz](https://github.com/dennybritz/reinforcement-learning/tree/master/PolicyGradient). I feel like the cliff walking environment in conjunction with the tabular case is not the best setting for testing the idea but I figured I'd still give it a try.

In [7]:
env = CliffWalkingEnv()

In [8]:
num_trials = 20
num_episodes = 4000
timeout = 2 * 60

configs = {
    'decay_factor': [1.0, 0.999],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
}

def job(seed, decay_factor, learning_rate):
    np.random.seed(seed=seed)
    
    class TimeoutException(Exception):
        pass

    def timeout_handler(signum, frame):
        raise TimeoutException
    
    tf.reset_default_graph()

    global_step = tf.Variable(0, name="global_step", trainable=False)
    loss_modifier = partial(apply_synthetic_grads, decay_factor=decay_factor, learning_rate=learning_rate)
    policy = Policy(loss_modifier=loss_modifier)
    value_function = ValueFunction()
    
    signal.signal(signal.SIGALRM, timeout_handler)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        signal.alarm(timeout)
        try:
            stats = reinforce(env, policy, value_function, num_episodes, discount_factor=1.0)
        except Exception, e:
            stats = plotting.EpisodeStats(
                episode_lengths=np.zeros(num_episodes),
                episode_rewards=np.full(num_episodes, np.nan))
        else:
            signal.alarm(0)
        stats.episode_rewards[stats.episode_lengths == 0.0] = np.nan
    
    return stats

for v in itertools.product(*configs.values()):
    config = dict(zip(configs.keys(), v))
    if config['decay_factor'] == 1.0:
        config['learning_rate'] = 0.01
    name = '{decay_factor}_{learning_rate}'.format(**config)
    filename = 'results/{}.npy'.format(name)
    if os.path.exists(filename):
        continue
    all_stats = Parallel(n_jobs=8)(
        [delayed(job)(s, **config) for s in np.random.randint(0, num_trials * 1000, num_trials)])
    np.save(filename, np.array([np.nanmean(s.episode_rewards) for s in all_stats]))

In [11]:
avg_rewards = []
keys = []
names = configs.keys()
for v in itertools.product(*configs.values()):
    config = dict(zip(configs.keys(), v))
    if config['decay_factor'] == 1.0:
        config['learning_rate'] = 0.01
    name = '{decay_factor}_{learning_rate}'.format(**config)
    filename = 'results/{}.npy'.format(name)
    avg_rewards.append(np.nanmean(np.load(filename)))
    keys.append(tuple(config[k] for k in names))
    
s = pd.Series(avg_rewards, index=pd.MultiIndex.from_tuples(keys, names=names))
s = s.reorder_levels(['decay_factor', 'learning_rate']).sort_index()

### Results

Below I'm showing mean rewards for several settings of the decay factor and learning rate of the gradient synthesizer. The numbers are averaged across 20 run. I actually tried more different tuples of hyperparameters but none of them worked any better.

In [12]:
s

decay_factor  learning_rate
0.999         0.01            -104.103758
              0.10             -85.140069
              0.20             -79.067548
              0.30             -79.587456
1.000         0.01             -68.744480
              0.01             -68.744480
              0.01             -68.744480
              0.01             -68.744480
dtype: float64

It's quite clear that the proposed approach fails to show any improvement over vanilla REINFORCE with baseline. As I only spent **very** limited amount of time playing around with the idea, this outcome may not be representative. I see several directions for the future work:

* Non-tabular case and different environments
* Careful design of the synthesizer network
* Optimization strategies for the synthesizer network